In [1]:
source = "ANN";
residual_alphas = ["UserItemBiases"];

In [2]:
using Flux
import BSON

In [3]:
using NBInclude
@nbinclude("Alpha.ipynb");

In [4]:
BSON.@load "../../data/alphas/$source/model.bson" m

In [5]:
df = cat(get_recommendee_list(), get_implicit_list());

In [6]:
X = zeros(Float32, num_items());
X[df.item] .= 1;

In [7]:
function get_input(X)
    # normalize for embeddings
    weight = sum(X .!= 0)
    if weight == 0
        weight = 1
    end
    X = X ./ weight

    return X
end;

In [8]:
p = exp.(m(get_input(X)))
p = p ./ sum(p);

In [9]:
model(items) = p[items]
write_recommendee_alpha(model)